In [14]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = list(glob.glob('../features/*.pkl'))
for l in sorted(fl):
    print("'{}'".format(l))

'../features/cnn_1_aug1_feat.pkl'
'../features/cnn_1_feat.pkl'
'../features/cnn_2_aug1_feat.pkl'
'../features/cnn_2_feat.pkl'
'../features/cnn_3_aug1_feat.pkl'
'../features/cnn_4_aug1_feat.pkl'
'../features/cnn_4_aug1_feat_add_early.pkl'
'../features/cnn_5_aug1_feat.pkl'
'../features/incept_1_feat.pkl'
'../features/incept_aug1_feat.pkl'
'../features/incept_aug2_feat.pkl'
'../features/incept_aug3_feat.pkl'
'../features/other_feat.pkl'
'../features/resnet_1_feat.pkl'
'../features/resnet_aug1_feat.pkl'
'../features/resnet_aug2_feat.pkl'
'../features/resnet_aug3_feat.pkl'
'../features/vgg16_1_feat.pkl'
'../features/vgg_aug1_feat.pkl'


In [15]:
def load_x_y():
    train_x,test_x=[],[]
    for f in fl:
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

(1604, 29) (8424, 29) (1604,)


In [16]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 1.0
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=30,
                        verbose_eval=100)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=w_test_pred
        print(submission.head())
        submission.to_csv('../results/weighted_all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

    
print('def done')

def done


In [17]:
cv_test(5,233,True)

[0]	train-logloss:0.640042	valid-logloss:0.64132
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.085731	valid-logloss:0.150959
Stopping. Best iteration:
[106]	train-logloss:0.081826	valid-logloss:0.150418

[0]	train-logloss:0.640553	valid-logloss:0.638461
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.095044	valid-logloss:0.107113
[200]	train-logloss:0.049733	valid-logloss:0.096345
[300]	train-logloss:0.029752	valid-logloss:0.092553
Stopping. Best iteration:
[363]	train-logloss:0.0228	valid-logloss:0.089514

[0]	train-logloss:0.639406	valid-logloss:0.643287
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.084865	valid

In [18]:
cv_test(5,42,True)

[0]	train-logloss:0.638593	valid-logloss:0.642662
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.080479	valid-logloss:0.17592
Stopping. Best iteration:
[120]	train-logloss:0.068915	valid-logloss:0.175046

[0]	train-logloss:0.639811	valid-logloss:0.643053
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.086819	valid-logloss:0.148765
[200]	train-logloss:0.046474	valid-logloss:0.140681
Stopping. Best iteration:
[231]	train-logloss:0.039025	valid-logloss:0.139519

[0]	train-logloss:0.639192	valid-logloss:0.641799
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.082562	valid-logloss:0.197983
Stopping. Best iteration:
[72]	t